In [79]:
import pandas as pd
import numpy as np
import time
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

## Graduation Rates

In [80]:
grad_rates_1011 = pd.read_excel("data/grad_rates/Grad_Rates_2010-11_cohort.xlsx", sheet_name=1, usecols="B:H", skiprows=6)
grad_rates_1112 = pd.read_excel("data/grad_rates/Grad_Rates_2011-12_cohort.xls", sheet_name=1, usecols="B:H", skiprows=5)
grad_rates_1213 = pd.read_excel("data/grad_rates/Grad_Rates_2012-13_cohort.xlsx", sheet_name=1, usecols="B:H", skiprows=6)
grad_rates_1314 = pd.read_excel("data/grad_rates/Grad_Rates_2013-14_cohort.xlsx", sheet_name=1, usecols="B:H", skiprows=6)
grad_rates_1011.head(3)

,School ID,School Name,Cohort,Subgroup,Students,Grads,Rate
0,31,Reti-Wrap Transition Center,2010-2011,All Students,28.0,5,0.178571
1,32,Family Support Center,2010-2011,All Students,3.0,s,s
2,101,John Bartram High School,2010-2011,All Students,281.0,203,0.72242


In [81]:
grad_rates_1112 = grad_rates_1112.rename(columns={'Subgrouop': 'Subgroup', '9th Grade Cohort': 'Cohort'})
grad_rates_1112 = grad_rates_1112[grad_rates_1112.Subgroup == 'All Students']

In [82]:
grad_rates = pd.concat([grad_rates_1011, grad_rates_1112, grad_rates_1213, grad_rates_1314]).reset_index(drop=True)
grad_rates = grad_rates.rename(columns={'Rate': 'GradRate'})
print(grad_rates.shape)
grad_rates.head()

(368, 7)


,School ID,School Name,Cohort,Subgroup,Students,Grads,GradRate
0,31,Reti-Wrap Transition Center,2010-2011,All Students,28,5,0.178571
1,32,Family Support Center,2010-2011,All Students,3,s,s
2,101,John Bartram High School,2010-2011,All Students,281,203,0.72242
3,102,West Philadelphia High School,2010-2011,All Students,182,123,0.675824
4,103,High School Of The Future,2010-2011,All Students,146,104,0.712329


In [83]:
# replace 's' with NaN
# 's' means supressed
grad_rates = grad_rates.replace('s', np.nan)

# if grad count is suprressed then suppress student count too
grad_rates['Students'] = grad_rates['Students'].where(~pd.isnull(grad_rates['Grads']), np.nan)

# convert School ID to string
grad_rates['School ID'] = grad_rates['School ID'].astype(str)

# all subgroups are the same (equals 'All Students')
grad_rates.drop('Subgroup', axis=1, inplace=True)

# remove school names. They'll be added later
grad_rates.drop('School Name', axis=1, inplace=True)

In [84]:
grad_rates.head()

,School ID,Cohort,Students,Grads,GradRate
0,31,2010-2011,28.0,5.0,0.178571
1,32,2010-2011,NaN,NaN,NaN
2,101,2010-2011,281.0,203.0,0.722420
3,102,2010-2011,182.0,123.0,0.675824
4,103,2010-2011,146.0,104.0,0.712329


In [85]:
# convert rows to columns
# get one row per school
grad_rates.set_index(['School ID', 'Cohort'], inplace=True)
grad_rates = grad_rates.unstack().reset_index()
grad_rates.columns = [' '.join(col).strip() for col in grad_rates.columns.values]
grad_rates.head()

,School ID,Students 2010-2011,Students 2011-2012,Students 2012-2013,Students 2013-2014,Grads 2010-2011,Grads 2011-2012,Grads 2012-2013,Grads 2013-2014,GradRate 2010-2011,GradRate 2011-2012,GradRate 2012-2013,GradRate 2013-2014
0,101,281.0,293.0,239.0,217.0,203.0,177.0,130.0,126.0,0.722420,0.604096,0.543933,0.580645
1,102,182.0,171.0,156.0,118.0,123.0,107.0,83.0,76.0,0.675824,0.625731,0.532051,0.644068
2,103,146.0,159.0,101.0,124.0,104.0,127.0,84.0,91.0,0.712329,0.798742,0.831683,0.733871
3,105,68.0,67.0,73.0,76.0,62.0,65.0,69.0,73.0,0.911765,0.970149,0.945205,0.960526
4,106,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [86]:
# df.columns = [' '.join(col).strip() for col in df.columns.values]
# df.head()

In [87]:
# get 4-year combined graduation rate
grad_rates['Students Total'] = grad_rates[['Students 2010-2011', 'Students 2011-2012', \
                                          'Students 2012-2013','Students 2013-2014']].sum(axis=1)
            
grad_rates['Grads Total'] = grad_rates[['Grads 2010-2011', 'Grads 2011-2012', \
                                        'Grads 2012-2013','Grads 2013-2014']].sum(axis=1)
            
grad_rates['GradRate Total'] = grad_rates['Grads Total'] / grad_rates['Students Total']
            
grad_rates.head()

,School ID,Students 2010-2011,Students 2011-2012,Students 2012-2013,Students 2013-2014,Grads 2010-2011,Grads 2011-2012,Grads 2012-2013,Grads 2013-2014,GradRate 2010-2011,GradRate 2011-2012,GradRate 2012-2013,GradRate 2013-2014,Students Total,Grads Total,GradRate Total
0,101,281.0,293.0,239.0,217.0,203.0,177.0,130.0,126.0,0.722420,0.604096,0.543933,0.580645,1030.0,636.0,0.617476
1,102,182.0,171.0,156.0,118.0,123.0,107.0,83.0,76.0,0.675824,0.625731,0.532051,0.644068,627.0,389.0,0.620415
2,103,146.0,159.0,101.0,124.0,104.0,127.0,84.0,91.0,0.712329,0.798742,0.831683,0.733871,530.0,406.0,0.766038
3,105,68.0,67.0,73.0,76.0,62.0,65.0,69.0,73.0,0.911765,0.970149,0.945205,0.960526,284.0,269.0,0.947183
4,106,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN


In [88]:
# subset columns
df = grad_rates[['School ID', 'GradRate 2010-2011', 'GradRate 2011-2012', 'GradRate 2012-2013', \
                 'GradRate 2013-2014', 'Students Total', 'Grads Total', 'GradRate Total']]

## Teacher Attendance Data

In [89]:
# teacher attendance
teach_attnd = pd.read_excel("data/Teacher_Attend_1516.xlsx", sheet_name=0)
# teach_attnd.set_index(['School ID'], inplace=True)

# convert School ID to string
teach_attnd['School ID'] = teach_attnd['School ID'].astype(str)

df2 = pd.merge(df, teach_attnd, how='outer', on='School ID')
df2.drop(['School Year', 'ULCS', 'SDP Avg'], axis=1, inplace=True)
df2.head()

,School ID,GradRate 2010-2011,GradRate 2011-2012,GradRate 2012-2013,GradRate 2013-2014,Students Total,Grads Total,GradRate Total,Teacher Attd %
0,101,0.722420,0.604096,0.543933,0.580645,1030.0,636.0,0.617476,93.5
1,102,0.675824,0.625731,0.532051,0.644068,627.0,389.0,0.620415,94.0
2,103,0.712329,0.798742,0.831683,0.733871,530.0,406.0,0.766038,91.4
3,105,0.911765,0.970149,0.945205,0.960526,284.0,269.0,0.947183,97.0
4,106,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN


## School Progress Report (SPR) Data

In [90]:
# read SPR data
spr = pd.read_excel('data/SPR_SY1617_Data/SPR_SY1617_School_Metric_Scores_20180206.xlsx', sheet_name=1)

# convert School ID to string
spr = spr.rename(columns={'SRC School ID': 'School ID'})
spr['School ID'] = spr['School ID'].astype(str)
# spr.set_index(['School ID', 'Report'], inplace=True)

spr['Full Address'] = spr['Street Address'] + ', ' + spr['City'] + ', ' + spr['State'] + ' ' + spr['Zip Code'].astype(str)
spr.drop(['Street Address', 'City', 'State', 'Zip Code', 'Phone Number', 'Fax Number'], axis=1, inplace=True)
spr.head(3)

,School,School ID,Report,Rpt Type Long,Governance,Turnaround Model,Enrollment,Website,Grades Served,Admissions Type,2016-17 Learning Network,2017-18 Learning Network,Notes,Overall Score,Overall Tier,Overall Peer Rank,Overall City Rank,Ach Score,Ach Tier,Ach Peer Rank,Ach City Rank,Prog Score,Prog Tier,Prog Peer Rank,Prog City Rank,Clim Score,Clim Tier,Clim Peer Rank,Clim City Rank,CC Score,CC Tier,CC Peer Rank,CC City Rank,K-2 Rdg Score,K-2 Rdg Denom,K-2 Rdg Pts Earn,K-2 Rdg Pts Poss,K-2 Rdg Pct Earn,K-2 Rdg Tier,PSSA ELA Prof/Adv Score,PSSA ELA Prof/Adv Denom,PSSA ELA Prof/Adv Pts Earn,PSSA ELA Prof/Adv Pts Poss,PSSA ELA Prof/Adv Pct Earn,PSSA ELA Prof/Adv Tier,PSSA GR3 Prof/Adv ELA Score,PSSA GR3 Prof/Adv ELA Denom,PSSA GR4-8 Prof/Adv ELA Score,PSSA GR4-8 Prof/Adv ELA Denom,PSSA ELA Adv Score,PSSA ELA Adv Denom,PSSA ELA Adv Pts Earn,PSSA ELA Adv Pts Poss,PSSA ELA Adv Pct Earn,PSSA ELA Adv Tier,PSSA GR3 Adv ELA Score,PSSA GR3 Adv ELA Denom,PSSA GR4-8 Adv ELA Score,PSSA GR4-8 Adv ELA Denom,PSSA Math Prof/Adv Score,PSSA Math Prof/Adv Denom,PSSA Math Prof/Adv Pts Earn,PSSA Math Prof/Adv Pts Poss,PSSA Math Prof/Adv Pct Earn,PSSA Math Prof/Adv Tier,PSSA Math Adv Score,PSSA Math Adv Denom,PSSA Math Adv Pts Earn,PSSA Math Adv Pts Poss,PSSA Math Adv Pct Earn,PSSA Math Adv Tier,PSSA Sci Prof/Adv Score,PSSA Sci Prof/Adv Denom,PSSA Sci Prof/Adv Pts Earn,PSSA Sci Prof/Adv Pts Poss,PSSA Sci Prof/Adv Pct Earn,PSSA Sci Prof/Adv Tier,PSSA Sci Adv Score,PSSA Sci Adv Denom,PSSA Sci Adv Pts Earn,PSSA Sci Adv Pts Poss,PSSA Sci Adv Pct Earn,PSSA Sci Adv Tier,Keystone Alg1 Prof/Adv Score,Keystone Alg1 Prof/Adv Denom,Keystone Alg1 Prof/Adv Pts Earn,Keystone Alg1 Prof/Adv Pts Poss,Keystone Alg1 Prof/Adv Pct Earn,Keystone Alg1 Prof/Adv Tier,Keystone Alg1 Adv Score,Keystone Alg1 Adv Denom,Keystone Alg1 Adv Pts Earn,Keystone Alg1 Adv Pts Poss,Keystone Alg1 Adv Pct Earn,Keystone Alg1 Adv Tier,Keystone Bio Prof/Adv Score,Keystone Bio Prof/Adv Denom,Keystone Bio Prof/Adv Pts Earn,Keystone Bio Prof/Adv Pts Poss,Keystone Bio Prof/Adv Pct Earn,Keystone Bio Prof/Adv Tier,Keystone Bio Adv Score,Keystone Bio Adv Denom,Keystone Bio Adv Pts Earn,Keystone Bio Adv Pts Poss,Keystone Bio Adv Pct Earn,Keystone Bio Adv Tier,Keystone Lit Prof/Adv Score,Keystone Lit Prof/Adv Denom,Keystone Lit Prof/Adv Pts Earn,Keystone Lit Prof/Adv Pts Poss,Keystone Lit Prof/Adv Pct Earn,Keystone Lit Prof/Adv Tier,Keystone Lit Adv Score,Keystone Lit Adv Denom,Keystone Lit Adv Pts Earn,Keystone Lit Adv Pts Poss,Keystone Lit Adv Pct Earn,Keystone Lit Adv Tier,ACCESS Prof Score,ACCESS Prof Denom,ACCESS Prof Pts Earn,ACCESS Prof Pts Poss,ACCESS Prof Pct Earn,ACCESS Prof Tier,PSSA Math AGI Score,PSSA Math AGI Pts Earn,PSSA Math AGI Pts Poss,PSSA Math AGI Pct Earn,PSSA Math AGI Tier,PSSA ELA AGI Score,PSSA ELA AGI Pts Earn,PSSA ELA AGI Pts Poss,PSSA ELA AGI Pct Earn,PSSA ELA AGI Tier,PSSA GR4 Sci AGI Score,PSSA GR4 Sci AGI Pts Earn,PSSA GR4 Sci AGI Pts Poss,PSSA GR4 Sci AGI Pct Earn,PSSA GR4 Sci AGI Tier,PSSA GR8 Sci AGI Score,PSSA GR8 Sci AGI Pts Earn,PSSA GR8 Sci AGI Pts Poss,PSSA GR8 Sci AGI Pct Earn,PSSA GR8 Sci AGI Tier,Keystone Alg1 AGI Score,Keystone Alg1 AGI Pts Earn,Keystone Alg1 AGI Pts Poss,Keystone Alg1 AGI Pct Earn,Keystone Alg1 AGI Tier,Keystone Bio AGI Score,Keystone Bio AGI Pts Earn,Keystone Bio AGI Pts Poss,Keystone Bio AGI Pct Earn,Keystone Bio AGI Tier,Keystone Lit AGI Score,Keystone Lit AGI Pts Earn,Keystone Lit AGI Pts Poss,Keystone Lit AGI Pct Earn,Keystone Lit AGI Tier,Stay On Track Score,Stay On Track Denom,Stay On Track Pts Earn,Stay On Track Pts Poss,Stay On Track Pct Earn,Stay On Track Tier,Back On Track Score,Back On Track Denom,Back On Track Pts Earn,Back On Track Pts Poss,Back On Track Pct Earn,Back On Track Tier,ACCESS Growth Score,ACCESS Growth Denom,ACCESS Growth Pts Earn,ACCESS Growth Pts Poss,ACCESS Growth Pct Earn,ACCESS Growth Tier,PSSA Math AGI (L20P) Score,PSSA Math AGI (L20P) Pts Earn,PSSA Math AGI (L20P) Pts Poss,PSSA Math AGI (L20P) Pct Earn,PSSA Math

# Get GPS Coordinates
For each school, use the address to find the Latitude and Longitude Coordinates. This will let us place the schools on a map.

In [91]:
geolocator = Nominatim()
cell_start = time.time()

lat = pd.Series({})
lon = pd.Series({})

for index, row in spr.iterrows():
    addr = row['Full Address']

    if row['School ID'] not in lat:
        # try geocoder 3 times
        for i in range(0,3):
            try:
                location = geolocator.geocode(addr)
                break
            except GeocoderTimedOut:
                print("Geocoder Timed Out. Address =", addr)
                continue

        try:
            latitude = location.latitude
            longitude = location.longitude
        except AttributeError:
            latitude = np.nan
            longitude = np.nan

        lat = pd.concat([lat, pd.Series({row['School ID']: latitude})])
        lon = pd.concat([lon, pd.Series({row['School ID']: longitude})])
    
    else:
        continue
        
print('Cell Runtime:', round(time.time() - cell_start), 'seconds')

Geocoder Timed Out. Address = 8500 Pickering St, Philadelphia, PA 19150
Cell Runtime: 152 seconds


In [93]:
print(lat.shape, sum(pd.isnull(lat)))
lat = lat.rename("Latitude")
lon = lon.rename("Longitude")

(294,) 47


In [94]:
lat.head()

101    39.927011
102    39.958587
103    39.974691
105    39.954262
110    39.955760
Name: Latitude, dtype: float64

In [95]:
temp1 = pd.merge(df2, spr, how='right', on='School ID')
temp2 = pd.merge(temp1, lat.to_frame(), how='left', left_on='School ID', right_index=True)
df3 = pd.merge(temp2, lon.to_frame(), how='left', left_on='School ID', right_index=True)
df3.head()

,School ID,GradRate 2010-2011,GradRate 2011-2012,GradRate 2012-2013,GradRate 2013-2014,Students Total,Grads Total,GradRate Total,Teacher Attd %,School,Report,Rpt Type Long,Governance,Turnaround Model,Enrollment,Website,Grades Served,Admissions Type,2016-17 Learning Network,2017-18 Learning Network,Notes,Overall Score,Overall Tier,Overall Peer Rank,Overall City Rank,Ach Score,Ach Tier,Ach Peer Rank,Ach City Rank,Prog Score,Prog Tier,Prog Peer Rank,Prog City Rank,Clim Score,Clim Tier,Clim Peer Rank,Clim City Rank,CC Score,CC Tier,CC Peer Rank,CC City Rank,K-2 Rdg Score,K-2 Rdg Denom,K-2 Rdg Pts Earn,K-2 Rdg Pts Poss,K-2 Rdg Pct Earn,K-2 Rdg Tier,PSSA ELA Prof/Adv Score,PSSA ELA Prof/Adv Denom,PSSA ELA Prof/Adv Pts Earn,PSSA ELA Prof/Adv Pts Poss,PSSA ELA Prof/Adv Pct Earn,PSSA ELA Prof/Adv Tier,PSSA GR3 Prof/Adv ELA Score,PSSA GR3 Prof/Adv ELA Denom,PSSA GR4-8 Prof/Adv ELA Score,PSSA GR4-8 Prof/Adv ELA Denom,PSSA ELA Adv Score,PSSA ELA Adv Denom,PSSA ELA Adv Pts Earn,PSSA ELA Adv Pts Poss,PSSA ELA Adv Pct Earn,PSSA ELA Adv Tier,PSSA GR3 Adv ELA Score,PSSA GR3 Adv ELA Denom,PSSA GR4-8 Adv ELA Score,PSSA GR4-8 Adv ELA Denom,PSSA Math Prof/Adv Score,PSSA Math Prof/Adv Denom,PSSA Math Prof/Adv Pts Earn,PSSA Math Prof/Adv Pts Poss,PSSA Math Prof/Adv Pct Earn,PSSA Math Prof/Adv Tier,PSSA Math Adv Score,PSSA Math Adv Denom,PSSA Math Adv Pts Earn,PSSA Math Adv Pts Poss,PSSA Math Adv Pct Earn,PSSA Math Adv Tier,PSSA Sci Prof/Adv Score,PSSA Sci Prof/Adv Denom,PSSA Sci Prof/Adv Pts Earn,PSSA Sci Prof/Adv Pts Poss,PSSA Sci Prof/Adv Pct Earn,PSSA Sci Prof/Adv Tier,PSSA Sci Adv Score,PSSA Sci Adv Denom,PSSA Sci Adv Pts Earn,PSSA Sci Adv Pts Poss,PSSA Sci Adv Pct Earn,PSSA Sci Adv Tier,Keystone Alg1 Prof/Adv Score,Keystone Alg1 Prof/Adv Denom,Keystone Alg1 Prof/Adv Pts Earn,Keystone Alg1 Prof/Adv Pts Poss,Keystone Alg1 Prof/Adv Pct Earn,Keystone Alg1 Prof/Adv Tier,Keystone Alg1 Adv Score,Keystone Alg1 Adv Denom,Keystone Alg1 Adv Pts Earn,Keystone Alg1 Adv Pts Poss,Keystone Alg1 Adv Pct Earn,Keystone Alg1 Adv Tier,Keystone Bio Prof/Adv Score,Keystone Bio Prof/Adv Denom,Keystone Bio Prof/Adv Pts Earn,Keystone Bio Prof/Adv Pts Poss,Keystone Bio Prof/Adv Pct Earn,Keystone Bio Prof/Adv Tier,Keystone Bio Adv Score,Keystone Bio Adv Denom,Keystone Bio Adv Pts Earn,Keystone Bio Adv Pts Poss,Keystone Bio Adv Pct Earn,Keystone Bio Adv Tier,Keystone Lit Prof/Adv Score,Keystone Lit Prof/Adv Denom,Keystone Lit Prof/Adv Pts Earn,Keystone Lit Prof/Adv Pts Poss,Keystone Lit Prof/Adv Pct Earn,Keystone Lit Prof/Adv Tier,Keystone Lit Adv Score,Keystone Lit Adv Denom,Keystone Lit Adv Pts Earn,Keystone Lit Adv Pts Poss,Keystone Lit Adv Pct Earn,Keystone Lit Adv Tier,ACCESS Prof Score,ACCESS Prof Denom,ACCESS Prof Pts Earn,ACCESS Prof Pts Poss,ACCESS Prof Pct Earn,ACCESS Prof Tier,PSSA Math AGI Score,PSSA Math AGI Pts Earn,PSSA Math AGI Pts Poss,PSSA Math AGI Pct Earn,PSSA Math AGI Tier,PSSA ELA AGI Score,PSSA ELA AGI Pts Earn,PSSA ELA AGI Pts Poss,PSSA ELA AGI Pct Earn,PSSA ELA AGI Tier,PSSA GR4 Sci AGI Score,PSSA GR4 Sci AGI Pts Earn,PSSA GR4 Sci AGI Pts Poss,PSSA GR4 Sci AGI Pct Earn,PSSA GR4 Sci AGI Tier,PSSA GR8 Sci AGI Score,PSSA GR8 Sci AGI Pts Earn,PSSA GR8 Sci AGI Pts Poss,PSSA GR8 Sci AGI Pct Earn,PSSA GR8 Sci AGI Tier,Keystone Alg1 AGI Score,Keystone Alg1 AGI Pts Earn,Keystone Alg1 AGI Pts Poss,Keystone Alg1 AGI Pct Earn,Keystone Alg1 AGI Tier,Keystone Bio AGI Score,Keystone Bio AGI Pts Earn,Keystone Bio AGI Pts Poss,Keystone Bio AGI Pct Earn,Keystone Bio AGI Tier,Keystone Lit AGI Score,Keystone Lit AGI Pts Earn,Keystone Lit AGI Pts Poss,Keystone Lit AGI Pct Earn,Keystone Lit AGI Tier,Stay On Track Score,Stay On Track Denom,Stay On Track Pts Earn,Stay On Track Pts Poss,Stay On Track Pct Earn,Stay On Track Tier,Back On Track Score,Back On Track Denom,Back On Track Pts Earn,Back On Track Pts Poss,Back On Track Pct Earn,Back On Track Tier,ACCESS Growth Score,ACCESS Growth Denom,ACCESS Growth Pts Earn,ACCESS Growth Pts Poss,ACCESS Growth Pct Earn,ACCESS Growt

## Demographics Data

In [96]:
# Excel file has 4 worksheets, one fo each school type (Elem, K-8, MS, HS)
demog_dict = pd.read_excel('data/SPR_SY1617_Peer_Group_Data/SPR_SY1617_Peer_Group_Demographics_20180122.xlsx', \
                             sheet_name=[1,2,3,4], usecols="B:H,J:M")
# create common column naming
demog_dict[1] = demog_dict[1].rename(columns={'ES Group 1 (18 schools)': 'School ID'})
demog_dict[2] = demog_dict[2].rename(columns={'K8 Group 1 (26 schools)': 'School ID'})
demog_dict[3] = demog_dict[3].rename(columns={'MS Group 1 (10 schools)': 'School ID'})
demog_dict[4] = demog_dict[4].rename(columns={'HS Group 1 (22 schools)': 'School ID'})

# mark school type
demog_dict[1]['Report'] = 'ES'
demog_dict[2]['Report'] = 'K8'
demog_dict[3]['Report'] = 'MS'
demog_dict[4]['Report'] = 'HS'

# combine/concat
demog = pd.concat([demog_dict[1], demog_dict[2], demog_dict[3], demog_dict[4]])

# remove rows with blank school names
# because of the way the Excel file is laid out, these are non-data rows
demog = demog[~pd.isnull(demog['Unnamed: 1'])]

# drop School Name since we have it in the other DF
demog.drop('Unnamed: 1', axis=1, inplace=True)

demog.head()

,School ID,Low-Income,Special Education,English Learner,Black/ African American,Hispanic/ Latino,White,Asian/ Pacific Islander,American Indian/ Alaska Native,Other/ Multiple Races,Report
0,526,0.893,0.114,0.229,0.204,0.692,0.024,-,-,0.08,ES
1,553,0.879,0.125,0.142,0.253,0.626,0.027,0.007,0.004,0.082,ES
2,559,0.862,0.174,0.082,0.287,0.496,0.086,0.008,0.002,0.12,ES
3,547,0.862,0.148,0.278,0.14,0.759,0.017,0.004,-,0.08,ES
4,544,0.826,0.136,0.129,0.289,0.523,0.081,0.001,0.003,0.103,ES


In [97]:
df4 = pd.merge(df3, demog, how='outer', on=['School ID', 'Report'])
df4.head()

,School ID,GradRate 2010-2011,GradRate 2011-2012,GradRate 2012-2013,GradRate 2013-2014,Students Total,Grads Total,GradRate Total,Teacher Attd %,School,Report,Rpt Type Long,Governance,Turnaround Model,Enrollment,Website,Grades Served,Admissions Type,2016-17 Learning Network,2017-18 Learning Network,Notes,Overall Score,Overall Tier,Overall Peer Rank,Overall City Rank,Ach Score,Ach Tier,Ach Peer Rank,Ach City Rank,Prog Score,Prog Tier,Prog Peer Rank,Prog City Rank,Clim Score,Clim Tier,Clim Peer Rank,Clim City Rank,CC Score,CC Tier,CC Peer Rank,CC City Rank,K-2 Rdg Score,K-2 Rdg Denom,K-2 Rdg Pts Earn,K-2 Rdg Pts Poss,K-2 Rdg Pct Earn,K-2 Rdg Tier,PSSA ELA Prof/Adv Score,PSSA ELA Prof/Adv Denom,PSSA ELA Prof/Adv Pts Earn,PSSA ELA Prof/Adv Pts Poss,PSSA ELA Prof/Adv Pct Earn,PSSA ELA Prof/Adv Tier,PSSA GR3 Prof/Adv ELA Score,PSSA GR3 Prof/Adv ELA Denom,PSSA GR4-8 Prof/Adv ELA Score,PSSA GR4-8 Prof/Adv ELA Denom,PSSA ELA Adv Score,PSSA ELA Adv Denom,PSSA ELA Adv Pts Earn,PSSA ELA Adv Pts Poss,PSSA ELA Adv Pct Earn,PSSA ELA Adv Tier,PSSA GR3 Adv ELA Score,PSSA GR3 Adv ELA Denom,PSSA GR4-8 Adv ELA Score,PSSA GR4-8 Adv ELA Denom,PSSA Math Prof/Adv Score,PSSA Math Prof/Adv Denom,PSSA Math Prof/Adv Pts Earn,PSSA Math Prof/Adv Pts Poss,PSSA Math Prof/Adv Pct Earn,PSSA Math Prof/Adv Tier,PSSA Math Adv Score,PSSA Math Adv Denom,PSSA Math Adv Pts Earn,PSSA Math Adv Pts Poss,PSSA Math Adv Pct Earn,PSSA Math Adv Tier,PSSA Sci Prof/Adv Score,PSSA Sci Prof/Adv Denom,PSSA Sci Prof/Adv Pts Earn,PSSA Sci Prof/Adv Pts Poss,PSSA Sci Prof/Adv Pct Earn,PSSA Sci Prof/Adv Tier,PSSA Sci Adv Score,PSSA Sci Adv Denom,PSSA Sci Adv Pts Earn,PSSA Sci Adv Pts Poss,PSSA Sci Adv Pct Earn,PSSA Sci Adv Tier,Keystone Alg1 Prof/Adv Score,Keystone Alg1 Prof/Adv Denom,Keystone Alg1 Prof/Adv Pts Earn,Keystone Alg1 Prof/Adv Pts Poss,Keystone Alg1 Prof/Adv Pct Earn,Keystone Alg1 Prof/Adv Tier,Keystone Alg1 Adv Score,Keystone Alg1 Adv Denom,Keystone Alg1 Adv Pts Earn,Keystone Alg1 Adv Pts Poss,Keystone Alg1 Adv Pct Earn,Keystone Alg1 Adv Tier,Keystone Bio Prof/Adv Score,Keystone Bio Prof/Adv Denom,Keystone Bio Prof/Adv Pts Earn,Keystone Bio Prof/Adv Pts Poss,Keystone Bio Prof/Adv Pct Earn,Keystone Bio Prof/Adv Tier,Keystone Bio Adv Score,Keystone Bio Adv Denom,Keystone Bio Adv Pts Earn,Keystone Bio Adv Pts Poss,Keystone Bio Adv Pct Earn,Keystone Bio Adv Tier,Keystone Lit Prof/Adv Score,Keystone Lit Prof/Adv Denom,Keystone Lit Prof/Adv Pts Earn,Keystone Lit Prof/Adv Pts Poss,Keystone Lit Prof/Adv Pct Earn,Keystone Lit Prof/Adv Tier,Keystone Lit Adv Score,Keystone Lit Adv Denom,Keystone Lit Adv Pts Earn,Keystone Lit Adv Pts Poss,Keystone Lit Adv Pct Earn,Keystone Lit Adv Tier,ACCESS Prof Score,ACCESS Prof Denom,ACCESS Prof Pts Earn,ACCESS Prof Pts Poss,ACCESS Prof Pct Earn,ACCESS Prof Tier,PSSA Math AGI Score,PSSA Math AGI Pts Earn,PSSA Math AGI Pts Poss,PSSA Math AGI Pct Earn,PSSA Math AGI Tier,PSSA ELA AGI Score,PSSA ELA AGI Pts Earn,PSSA ELA AGI Pts Poss,PSSA ELA AGI Pct Earn,PSSA ELA AGI Tier,PSSA GR4 Sci AGI Score,PSSA GR4 Sci AGI Pts Earn,PSSA GR4 Sci AGI Pts Poss,PSSA GR4 Sci AGI Pct Earn,PSSA GR4 Sci AGI Tier,PSSA GR8 Sci AGI Score,PSSA GR8 Sci AGI Pts Earn,PSSA GR8 Sci AGI Pts Poss,PSSA GR8 Sci AGI Pct Earn,PSSA GR8 Sci AGI Tier,Keystone Alg1 AGI Score,Keystone Alg1 AGI Pts Earn,Keystone Alg1 AGI Pts Poss,Keystone Alg1 AGI Pct Earn,Keystone Alg1 AGI Tier,Keystone Bio AGI Score,Keystone Bio AGI Pts Earn,Keystone Bio AGI Pts Poss,Keystone Bio AGI Pct Earn,Keystone Bio AGI Tier,Keystone Lit AGI Score,Keystone Lit AGI Pts Earn,Keystone Lit AGI Pts Poss,Keystone Lit AGI Pct Earn,Keystone Lit AGI Tier,Stay On Track Score,Stay On Track Denom,Stay On Track Pts Earn,Stay On Track Pts Poss,Stay On Track Pct Earn,Stay On Track Tier,Back On Track Score,Back On Track Denom,Back On Track Pts Earn,Back On Track Pts Poss,Back On Track Pct Earn,Back On Track Tier,ACCESS Growth Score,ACCESS Growth Denom,ACCESS Growth Pts Earn,ACCESS Growth Pts Poss,ACCESS Growth Pct Earn,ACCESS Growt

## Serious Incidents (aka Safety)

In [98]:
si_1314 = pd.read_csv('data/Serious_Incidents/Serious_Incidents_2013-14.TXT')
si_1415 = pd.read_excel('data/Serious_Incidents/Serious_Incidents_2014-15.xlsx', sheet_name=0)
si_1415.rename(columns={'ULCS Code':'ULCS_NO', 
                        'School Year':'SCHOOL_YEAR', 
                        'Incident Type':'INCIDENT_TYPE',
                        'Incident Count':'INCIDENT_COUNT',
                        'School ID':'SCHOOL_ID'}, inplace=True)

si_1516 = pd.read_excel('data/Serious_Incidents/Serious_Incidents_2015-16.xls', sheet_name=0)
si_1617 = pd.read_excel('data/Serious_Incidents/Serious_Incidents_2016-17.xlsx', sheet_name=0)

si = pd.concat([si_1314, si_1415, si_1516, si_1617]).reset_index(drop=True)
si.drop('ULCS_NO', axis=1, inplace=True)
si['INCIDENT_TYPE'] = si['INCIDENT_TYPE'].str.upper()
si['INCIDENT_TYPE'].replace('SUICIDE - ATTEMPTS & THREATS', 'SUICIDE - ATTEMPTS AND THREATS', inplace=True)
si['INCIDENT_TYPE'].replace('DRUGS & ALCOHOL OFFENSE', 'DRUG AND ALCOHOL OFFENSES', inplace=True)
si['INCIDENT_TYPE'].replace('DRUG & ALCOHOL OFFENSES', 'DRUG AND ALCOHOL OFFENSES', inplace=True)
si['INCIDENT_TYPE'].replace('THREATS', 'THREAT', inplace=True)
si['INCIDENT_TYPE'].replace('THEFTS', 'THEFT', inplace=True)
si['INCIDENT_TYPE'].replace('BOMB SCARES', 'BOMB SCARE', inplace=True)
si['INCIDENT_TYPE'].replace('FIRES AND FALSE ALARMS', 'FIRE - INCENDIARY', inplace=True)
si['INCIDENT_TYPE'].replace('THREATS - VERBAL AND WRITTEN', 'THREAT', inplace=True)
si['INCIDENT_TYPE'].replace('WEAPONS OFFENSES', 'WEAPONS', inplace=True)

# convert School ID to string
si['SCHOOL_ID'] = si['SCHOOL_ID'].astype(str)

si.head()

,INCIDENT_COUNT,INCIDENT_TYPE,SCHOOL_ID,SCHOOL_YEAR
0,1,HARASSMENT,526,2013-2014
1,5,ASSAULTS,534,2013-2014
2,8,THREAT,535,2013-2014
3,0,TAKE OFF,537,2013-2014
4,0,DRUG AND ALCOHOL OFFENSES,545,2013-2014


In [99]:
si2 = pd.pivot_table(si, values='INCIDENT_COUNT', 
                   index='SCHOOL_ID', 
                   columns=['INCIDENT_TYPE'], 
                   aggfunc=np.sum).fillna(0).reset_index()

# convert School ID to string
si2['SCHOOL_ID'] = si2['SCHOOL_ID'].astype(str)


pd.set_option('display.max_columns', 500)
df5 = pd.merge(df4, si2, how='left', left_on='School ID', right_on='SCHOOL_ID')
df5.drop('SCHOOL_ID', axis=1, inplace=True)

In [100]:
from sklearn import preprocessing

# scale 'recipient_cfscore' to be 0-1 so it matches our ideology score above
min_max_scaler = preprocessing.MinMaxScaler(feature_range=(0,100))

df5['Safety_incidents'] = df5['ASSAULTS'] + df5['BULLYING'] + df5['HARASSMENT'] + df5['BURGLARY'] + df5['ROBBERY'] + df5['THEFT']

null_index = df5['Safety_incidents'].isnull() 

df5.loc[~null_index, 'SafetyScore'] = 100 - min_max_scaler.fit_transform(np.array(df5.loc[~null_index, 'Safety_incidents']).reshape(-1, 1))
df5.SafetyScore = df5.SafetyScore.round()


df5.head()

,School ID,GradRate 2010-2011,GradRate 2011-2012,GradRate 2012-2013,GradRate 2013-2014,Students Total,Grads Total,GradRate Total,Teacher Attd %,School,Report,Rpt Type Long,Governance,Turnaround Model,Enrollment,Website,Grades Served,Admissions Type,2016-17 Learning Network,2017-18 Learning Network,Notes,Overall Score,Overall Tier,Overall Peer Rank,Overall City Rank,Ach Score,Ach Tier,Ach Peer Rank,Ach City Rank,Prog Score,Prog Tier,Prog Peer Rank,Prog City Rank,Clim Score,Clim Tier,Clim Peer Rank,Clim City Rank,CC Score,CC Tier,CC Peer Rank,CC City Rank,K-2 Rdg Score,K-2 Rdg Denom,K-2 Rdg Pts Earn,K-2 Rdg Pts Poss,K-2 Rdg Pct Earn,K-2 Rdg Tier,PSSA ELA Prof/Adv Score,PSSA ELA Prof/Adv Denom,PSSA ELA Prof/Adv Pts Earn,PSSA ELA Prof/Adv Pts Poss,PSSA ELA Prof/Adv Pct Earn,PSSA ELA Prof/Adv Tier,PSSA GR3 Prof/Adv ELA Score,PSSA GR3 Prof/Adv ELA Denom,PSSA GR4-8 Prof/Adv ELA Score,PSSA GR4-8 Prof/Adv ELA Denom,PSSA ELA Adv Score,PSSA ELA Adv Denom,PSSA ELA Adv Pts Earn,PSSA ELA Adv Pts Poss,PSSA ELA Adv Pct Earn,PSSA ELA Adv Tier,PSSA GR3 Adv ELA Score,PSSA GR3 Adv ELA Denom,PSSA GR4-8 Adv ELA Score,PSSA GR4-8 Adv ELA Denom,PSSA Math Prof/Adv Score,PSSA Math Prof/Adv Denom,PSSA Math Prof/Adv Pts Earn,PSSA Math Prof/Adv Pts Poss,PSSA Math Prof/Adv Pct Earn,PSSA Math Prof/Adv Tier,PSSA Math Adv Score,PSSA Math Adv Denom,PSSA Math Adv Pts Earn,PSSA Math Adv Pts Poss,PSSA Math Adv Pct Earn,PSSA Math Adv Tier,PSSA Sci Prof/Adv Score,PSSA Sci Prof/Adv Denom,PSSA Sci Prof/Adv Pts Earn,PSSA Sci Prof/Adv Pts Poss,PSSA Sci Prof/Adv Pct Earn,PSSA Sci Prof/Adv Tier,PSSA Sci Adv Score,PSSA Sci Adv Denom,PSSA Sci Adv Pts Earn,PSSA Sci Adv Pts Poss,PSSA Sci Adv Pct Earn,PSSA Sci Adv Tier,Keystone Alg1 Prof/Adv Score,Keystone Alg1 Prof/Adv Denom,Keystone Alg1 Prof/Adv Pts Earn,Keystone Alg1 Prof/Adv Pts Poss,Keystone Alg1 Prof/Adv Pct Earn,Keystone Alg1 Prof/Adv Tier,Keystone Alg1 Adv Score,Keystone Alg1 Adv Denom,Keystone Alg1 Adv Pts Earn,Keystone Alg1 Adv Pts Poss,Keystone Alg1 Adv Pct Earn,Keystone Alg1 Adv Tier,Keystone Bio Prof/Adv Score,Keystone Bio Prof/Adv Denom,Keystone Bio Prof/Adv Pts Earn,Keystone Bio Prof/Adv Pts Poss,Keystone Bio Prof/Adv Pct Earn,Keystone Bio Prof/Adv Tier,Keystone Bio Adv Score,Keystone Bio Adv Denom,Keystone Bio Adv Pts Earn,Keystone Bio Adv Pts Poss,Keystone Bio Adv Pct Earn,Keystone Bio Adv Tier,Keystone Lit Prof/Adv Score,Keystone Lit Prof/Adv Denom,Keystone Lit Prof/Adv Pts Earn,Keystone Lit Prof/Adv Pts Poss,Keystone Lit Prof/Adv Pct Earn,Keystone Lit Prof/Adv Tier,Keystone Lit Adv Score,Keystone Lit Adv Denom,Keystone Lit Adv Pts Earn,Keystone Lit Adv Pts Poss,Keystone Lit Adv Pct Earn,Keystone Lit Adv Tier,ACCESS Prof Score,ACCESS Prof Denom,ACCESS Prof Pts Earn,ACCESS Prof Pts Poss,ACCESS Prof Pct Earn,ACCESS Prof Tier,PSSA Math AGI Score,PSSA Math AGI Pts Earn,PSSA Math AGI Pts Poss,PSSA Math AGI Pct Earn,PSSA Math AGI Tier,PSSA ELA AGI Score,PSSA ELA AGI Pts Earn,PSSA ELA AGI Pts Poss,PSSA ELA AGI Pct Earn,PSSA ELA AGI Tier,PSSA GR4 Sci AGI Score,PSSA GR4 Sci AGI Pts Earn,PSSA GR4 Sci AGI Pts Poss,PSSA GR4 Sci AGI Pct Earn,PSSA GR4 Sci AGI Tier,PSSA GR8 Sci AGI Score,PSSA GR8 Sci AGI Pts Earn,PSSA GR8 Sci AGI Pts Poss,PSSA GR8 Sci AGI Pct Earn,PSSA GR8 Sci AGI Tier,Keystone Alg1 AGI Score,Keystone Alg1 AGI Pts Earn,Keystone Alg1 AGI Pts Poss,Keystone Alg1 AGI Pct Earn,Keystone Alg1 AGI Tier,Keystone Bio AGI Score,Keystone Bio AGI Pts Earn,Keystone Bio AGI Pts Poss,Keystone Bio AGI Pct Earn,Keystone Bio AGI Tier,Keystone Lit AGI Score,Keystone Lit AGI Pts Earn,Keystone Lit AGI Pts Poss,Keystone Lit AGI Pct Earn,Keystone Lit AGI Tier,Stay On Track Score,Stay On Track Denom,Stay On Track Pts Earn,Stay On Track Pts Poss,Stay On Track Pct Earn,Stay On Track Tier,Back On Track Score,Back On Track Denom,Back On Track Pts Earn,Back On Track Pts Poss,Back On Track Pct Earn,Back On Track Tier,ACCESS Growth Score,ACCESS Growth Denom,ACCESS Growth Pts Earn,ACCESS Growth Pts Poss,ACCESS Growth Pct Earn,ACCESS Growt

# SPR

In [110]:
spr = pd.read_excel('data/SPR_SY1617_Data/SPR_YearoverYear_20180129.xlsx', \
                             sheet_name="Sheet1", usecols="B:H,J:M", skiprows=9, header=0)
spr['School ID'] = spr['School ID'].astype(str)
spr.head(5)

,School ID,School Name,2017-18 Learning Network,Report Type (School Level),YOY Change (2015-16 to 2016-17),ACHIEVEMT Domain Score,PROGRESS Domain Score,COLL&CAR Domain Score,OVERALL SPR Score,ACHIEVEMT Domain Score.1,PROGRESS Domain Score.1
0,101,John Bartram High School,Neighborhood Network 1,HS,5.0,0.0,28.0,5.0,14,4.0,17.0
1,102,West Philadelphia High School,Turnaround Network,HS,8.0,0.0,44.0,12.0,22,0.0,24.0
2,103,High School of the Future,Neighborhood Network 2,HS,-1.0,3.0,27.0,17.0,23,2.0,24.0
3,105,Paul Robeson High School for Human Services,Neighborhood Network 2,HS,-8.0,0.0,60.0,55.0,41,6.0,76.0
4,106,Communication Tech HS^,Closed,HS,NaN,NaN,NaN,NaN,Closed,NaN,NaN


In [111]:
spr = spr[(spr["2017-18 Learning Network"]!="Closed")]
spr.head(5)

,School ID,School Name,2017-18 Learning Network,Report Type (School Level),YOY Change (2015-16 to 2016-17),ACHIEVEMT Domain Score,PROGRESS Domain Score,COLL&CAR Domain Score,OVERALL SPR Score,ACHIEVEMT Domain Score.1,PROGRESS Domain Score.1
0,101,John Bartram High School,Neighborhood Network 1,HS,5.0,0.0,28.0,5.0,14,4.0,17.0
1,102,West Philadelphia High School,Turnaround Network,HS,8.0,0.0,44.0,12.0,22,0.0,24.0
2,103,High School of the Future,Neighborhood Network 2,HS,-1.0,3.0,27.0,17.0,23,2.0,24.0
3,105,Paul Robeson High School for Human Services,Neighborhood Network 2,HS,-8.0,0.0,60.0,55.0,41,6.0,76.0
6,110,William L. Sayre High School,Neighborhood Network 2,HS,0.0,0.0,13.0,5.0,8,0.0,13.0


In [114]:
t1 = pd.merge(spr, lat.to_frame(), how='left', left_on='School ID', right_index=True)
spr = pd.merge(t1, lon.to_frame(), how='left', left_on='School ID', right_index=True)
df6 = pd.merge(spr, df5, how='left', left_on='School ID', right_index=True)
df6.head(100)

,School ID_x,School Name,2017-18 Learning Network_x,Report Type (School Level),YOY Change (2015-16 to 2016-17),ACHIEVEMT Domain Score,PROGRESS Domain Score,COLL&CAR Domain Score,OVERALL SPR Score,ACHIEVEMT Domain Score.1,PROGRESS Domain Score.1,Latitude_x,Longitude_x,Latitude_y,Longitude_y,School ID_y,GradRate 2010-2011,GradRate 2011-2012,GradRate 2012-2013,GradRate 2013-2014,Students Total,Grads Total,GradRate Total,Teacher Attd %,School,Report,Rpt Type Long,Governance,Turnaround Model,Enrollment,Website,Grades Served,Admissions Type,2016-17 Learning Network,2017-18 Learning Network_y,Notes,Overall Score,Overall Tier,Overall Peer Rank,Overall City Rank,Ach Score,Ach Tier,Ach Peer Rank,Ach City Rank,Prog Score,Prog Tier,Prog Peer Rank,Prog City Rank,Clim Score,Clim Tier,Clim Peer Rank,Clim City Rank,CC Score,CC Tier,CC Peer Rank,CC City Rank,K-2 Rdg Score,K-2 Rdg Denom,K-2 Rdg Pts Earn,K-2 Rdg Pts Poss,K-2 Rdg Pct Earn,K-2 Rdg Tier,PSSA ELA Prof/Adv Score,PSSA ELA Prof/Adv Denom,PSSA ELA Prof/Adv Pts Earn,PSSA ELA Prof/Adv Pts Poss,PSSA ELA Prof/Adv Pct Earn,PSSA ELA Prof/Adv Tier,PSSA GR3 Prof/Adv ELA Score,PSSA GR3 Prof/Adv ELA Denom,PSSA GR4-8 Prof/Adv ELA Score,PSSA GR4-8 Prof/Adv ELA Denom,PSSA ELA Adv Score,PSSA ELA Adv Denom,PSSA ELA Adv Pts Earn,PSSA ELA Adv Pts Poss,PSSA ELA Adv Pct Earn,PSSA ELA Adv Tier,PSSA GR3 Adv ELA Score,PSSA GR3 Adv ELA Denom,PSSA GR4-8 Adv ELA Score,PSSA GR4-8 Adv ELA Denom,PSSA Math Prof/Adv Score,PSSA Math Prof/Adv Denom,PSSA Math Prof/Adv Pts Earn,PSSA Math Prof/Adv Pts Poss,PSSA Math Prof/Adv Pct Earn,PSSA Math Prof/Adv Tier,PSSA Math Adv Score,PSSA Math Adv Denom,PSSA Math Adv Pts Earn,PSSA Math Adv Pts Poss,PSSA Math Adv Pct Earn,PSSA Math Adv Tier,PSSA Sci Prof/Adv Score,PSSA Sci Prof/Adv Denom,PSSA Sci Prof/Adv Pts Earn,PSSA Sci Prof/Adv Pts Poss,PSSA Sci Prof/Adv Pct Earn,PSSA Sci Prof/Adv Tier,PSSA Sci Adv Score,PSSA Sci Adv Denom,PSSA Sci Adv Pts Earn,PSSA Sci Adv Pts Poss,PSSA Sci Adv Pct Earn,PSSA Sci Adv Tier,Keystone Alg1 Prof/Adv Score,Keystone Alg1 Prof/Adv Denom,Keystone Alg1 Prof/Adv Pts Earn,Keystone Alg1 Prof/Adv Pts Poss,Keystone Alg1 Prof/Adv Pct Earn,Keystone Alg1 Prof/Adv Tier,Keystone Alg1 Adv Score,Keystone Alg1 Adv Denom,Keystone Alg1 Adv Pts Earn,Keystone Alg1 Adv Pts Poss,Keystone Alg1 Adv Pct Earn,Keystone Alg1 Adv Tier,Keystone Bio Prof/Adv Score,Keystone Bio Prof/Adv Denom,Keystone Bio Prof/Adv Pts Earn,Keystone Bio Prof/Adv Pts Poss,Keystone Bio Prof/Adv Pct Earn,Keystone Bio Prof/Adv Tier,Keystone Bio Adv Score,Keystone Bio Adv Denom,Keystone Bio Adv Pts Earn,Keystone Bio Adv Pts Poss,Keystone Bio Adv Pct Earn,Keystone Bio Adv Tier,Keystone Lit Prof/Adv Score,Keystone Lit Prof/Adv Denom,Keystone Lit Prof/Adv Pts Earn,Keystone Lit Prof/Adv Pts Poss,Keystone Lit Prof/Adv Pct Earn,Keystone Lit Prof/Adv Tier,Keystone Lit Adv Score,Keystone Lit Adv Denom,Keystone Lit Adv Pts Earn,Keystone Lit Adv Pts Poss,Keystone Lit Adv Pct Earn,Keystone Lit Adv Tier,ACCESS Prof Score,ACCESS Prof Denom,ACCESS Prof Pts Earn,ACCESS Prof Pts Poss,ACCESS Prof Pct Earn,ACCESS Prof Tier,PSSA Math AGI Score,PSSA Math AGI Pts Earn,PSSA Math AGI Pts Poss,PSSA Math AGI Pct Earn,PSSA Math AGI Tier,PSSA ELA AGI Score,PSSA ELA AGI Pts Earn,PSSA ELA AGI Pts Poss,PSSA ELA AGI Pct Earn,PSSA ELA AGI Tier,PSSA GR4 Sci AGI Score,PSSA GR4 Sci AGI Pts Earn,PSSA GR4 Sci AGI Pts Poss,PSSA GR4 Sci AGI Pct Earn,PSSA GR4 Sci AGI Tier,PSSA GR8 Sci AGI Score,PSSA GR8 Sci AGI Pts Earn,PSSA GR8 Sci AGI Pts Poss,PSSA GR8 Sci AGI Pct Earn,PSSA GR8 Sci AGI Tier,Keystone Alg1 AGI Score,Keystone Alg1 AGI Pts Earn,Keystone Alg1 AGI Pts Poss,Keystone Alg1 AGI Pct Earn,Keystone Alg1 AGI Tier,Keystone Bio AGI Score,Keystone Bio AGI Pts Earn,Keystone Bio AGI Pts Poss,Keystone Bio AGI Pct Earn,Keystone Bio AGI Tier,Keystone Lit AGI Score,Keystone Lit AGI Pts Earn,Keystone Lit AGI Pts Poss,Keystone Lit AGI Pct Earn,Keystone Lit AGI Tier,Stay On Track Score,Stay On Track Denom,Stay On Track Pts Earn,Stay On Track Pts Po

# Export Data

In [113]:
# save data
df6.to_csv("philly_school_data.csv", sep='|', index=False)

In [21]:
# print(min(lat), max(lat))
# print(min(lon), max(lon))